In [ ]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

# llm = OpenAI()
chat = ChatOpenAI(
	temperature=0.1,	
	streaming=True,
	callbacks=[
		StreamingStdOutCallbackHandler()
	]
)

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

# a
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [3]:
from langchain.schema import SystemMessage, AIMessage, HumanMessage

messages = [
	SystemMessage(
		content = "당신은 지리학 전문가입니다. 당신은 한국어로만 대답할 수 있습니다.",
	),
	AIMessage(
		content = "안녕하세요, 질문하신 곳에 대해 설명드립니다."
	),
	HumanMessage(
		content = "안녕하세요, 대한민국 성남시 분당구에 대해 설명해주세요."
	),
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요. 대한민국 경기도 성남시 분당구는 경기도 북부에 위치한 구로, 주변에는 성남시 수정구, 용인시 수지구, 하남시가 인접해 있습니다. 분당구는 주거지역과 상업지역이 조화롭게 어우러져 있으며, IT 기업들의 본사나 연구소가 밀집해 있는 지역으로 유명합니다. 또한, 분당구는 교육시설이 잘 발달되어 있어 학교가 많이 위치해 있습니다. 분당구는 깨끗하고 안전한 도시로 유명하며, 많은 사람들이 거주하고 있는 인기 있는 지역입니다.')

In [4]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
	"안녕하세요, {country}의 {area} 에 대해 설명해주세요."
)

prompt = template.format(country="대한민국", area="영종도")
chat.predict(prompt)

'영종도는 대한민국 인천광역시에 위치한 섬으로, 인천국제공항이 위치해 있습니다. 영종도는 인천의 중심부에 위치하고 있어 교통이 편리하며, 인천국제공항을 이용하는 여행객들에게 편리한 숙박시설과 다양한 관광 명소가 있습니다.\n\n영종도는 자연환경이 아름다운 곳으로 유명하며, 해안가를 따라 산책로가 조성되어 있어 풍경을 감상하며 산책을 즐길 수 있습니다. 또한, 영종도 주변에는 해수욕장이 있어 여름철에는 해변에서 수영을 즐길 수도 있습니다.\n\n또한, 영종도에는 다양한 레저 시설과 문화 시설이 있어 관광객들에게 다채로운 즐길 거리를 제공하고 있습니다. 인천국제공항을 이용하는 여행객들이나 근교에서 휴식을 취하고 싶은 이들에게 추천할 만한 여행지입니다.'

In [5]:
template2 = ChatPromptTemplate.from_messages(
	[
		("system", "당신은 지리학 전문가입니다. 당신은 {language}로만 대답할 수 있습니다."),
		("ai", "안녕하세요, 지리학 전문가 {name} 입니다."),
		("human", "안녕하세요, {country}의 {area}에 대해 설명해주세요.")
	]
)

prompt2 = template2.format_messages(
	language="Korean",
	name="이박사",
	country="대한민국",
	area="인천국제공항"
)

chat.predict_messages(prompt2)

AIMessage(content='인천국제공항은 대한민국의 주요 국제공항으로, 수도권 인천광역시 중구에 위치해 있습니다. 이 공항은 대한민국의 대표적인 항공 허브로서 국내선과 국제선을 운영하고 있습니다. 인천국제공항은 세계적으로도 중요한 공항으로 인정받고 있으며, 최첨단 시설과 편의시설을 갖추고 있습니다. 또한, 인천국제공항은 인천공항철도와 연결되어 있어 교통이 매우 편리합니다.')

In [11]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
	
	def parse(self, text):
		items = text.strip().split(", ")
		return list(map(str.strip, items))
	

cop = CommaOutputParser()
# cop.parse('Hi, Hello, How, are, you, ?')

template3 = ChatPromptTemplate.from_messages([
	("system", "당신은 어떤 질문을 받든 모든 대답을 콤마(,) 로 구분된 리스트로 합니다. 응답의 최대 길이는 {max_items} 입니다. 다른 형태로는 대답하지 않습니다. 또한 한글로만 답변합니다."),
	("human", "{question}"),
])

prompt3 = template3.format_messages(max_items=10, question="태양계에 대해서 알려주세요.")
result = chat.predict_messages(prompt3)
cop.parse(result.content)

['태양', '수성', '금성', '지구', '화성', '목성', '토성', '천왕성', '해왕성', '소행성들로 이루어져 있습니다.']

In [ ]:
# chain, LangChain Expression Language

chain = template3 | chat | cop
chain.invoke({
	"max_items": 5,
	"question": "포켓몬의 종류에 대해서 알려주세요."
})

['피카츄', '파이리', '꼬부기', '이상해씨', '꼬마돌']

In [14]:
chef_prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 월드 클래스 요리사입니다. 당신은 요청받은 지역의 음식 중 하나를 선택해서 쉽게 따라할 수 있는 step-by-step 레시피를 제작합니다."),
	("human", "저는 {cuisine} 음식을 만들고 싶어요."),
])
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 채식 전문 요리사입니다. 일반 레시피를 채식 요리로 바꾸는데 전문가입니다."),
	("human", "{recipe}"),
])
veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain
final_chain.invoke({
	"cuisine": "korean"
})

AIMessage(content='떡볶이 레시피를 채식 버전으로 변환해보겠습니다.\n\n**채식 떡볶이 레시피**\n\n**재료:**\n- 떡 2컵\n- 양배추 1/4개\n- 양파 1/2개\n- 대파 1대\n- 고추장 3큰술\n- 고춧가루 1큰술\n- 설탕 1큰술\n- 물 2컵\n- 국물용 다시마 5x5cm 크기 1조각\n- 식용유 1큰술\n\n**채식 대체 재료:**\n- 대파 대신 부추나 파프리카를 사용할 수 있습니다.\n- 국물용 멸치 대신 김 또는 해조류를 사용할 수 있습니다.\n\n**만들기:**\n1. 떡을 물에 불려서 준비해주세요.\n2. 양배추, 양파, 대파 대신 부추나 파프리카를 채 썰어주세요.\n3. 냄비에 물과 다시마를 넣고 끓여 국물을 만들어주세요.\n4. 끓는 물에 고추장, 고춧가루, 설탕을 넣고 잘 섞어주세요.\n5. 양배추, 양파 대신 부추나 파프리카를 넣고 끓여주세요.\n6. 떡과 국물을 넣고 끓여주세요.\n7. 대파 대신 부추나 파프리카를 넣고 한 번 더 끓여주세요.\n8. 식용유를 조금 넣어 윤기를 내면 완성입니다.\n\n이렇게 간단한 재료로 맛있는 채식 떡볶이를 만들 수 있어요. 맛있게 즐기세요!')